# Advanced Fairness Metrics Tutorial

In this tutorial, you'll learn about:

1. All fairness metrics in detail
2. Pre-training vs post-training metrics
3. When to use each metric
4. Trade-offs between metrics
5. Interpreting complex scenarios

**Time**: ~20 minutes  
**Prerequisites**: Complete Tutorial 01 (Quick Start)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from justiceai import FairnessEvaluator
from justiceai.core.metrics.calculator import FairnessCalculator

np.random.seed(42)
print("✓ Imports successful!")

## Part 1: Understanding Pre-training Metrics

Pre-training metrics analyze the dataset **before** model training to identify potential sources of bias.

### 1.1 Class Balance

Checks if the target variable is balanced across sensitive attribute groups.

In [ ]:
# Create dataset with class imbalance across groups
n_samples = 1000

# Create two groups with different positive rates
group_a_size = 600
group_b_size = 400

# Group A: 30% positive rate
y_a = np.random.choice([0, 1], size=group_a_size, p=[0.7, 0.3])
# Group B: 50% positive rate (imbalanced!)
y_b = np.random.choice([0, 1], size=group_b_size, p=[0.5, 0.5])

y = np.concatenate([y_a, y_b])
sensitive_attr = np.array(['A'] * group_a_size + ['B'] * group_b_size)

print("Class Balance Analysis:")
print(f"Group A positive rate: {y_a.mean():.3f}")
print(f"Group B positive rate: {y_b.mean():.3f}")
print(f"Difference: {abs(y_a.mean() - y_b.mean()):.3f}")
print("\nThis indicates potential bias in the data collection or historical outcomes.")

### 1.2 Concept Balance (Correlation)

Measures if the sensitive attribute correlates with the target outcome.

In [ ]:
# Calculate correlation
sensitive_numeric = (sensitive_attr == 'B').astype(int)
correlation = np.corrcoef(sensitive_numeric, y)[0, 1]

print(f"Correlation between sensitive attribute and target: {correlation:.3f}")
print("\nHigh correlation indicates the sensitive attribute")
print("is predictive of the outcome, which may lead to unfair models.")

## Part 2: Post-training Metrics Deep Dive

Post-training metrics analyze **model predictions** to measure fairness.

In [ ]:
# Create a more realistic dataset
X, y = make_classification(
    n_samples=2000,
    n_features=15,
    n_informative=10,
    n_redundant=3,
    n_classes=2,
    weights=[0.6, 0.4],
    random_state=42
)

# Create sensitive attribute with correlation to outcome
gender = np.random.choice(['Male', 'Female'], size=2000, p=[0.52, 0.48])
male_mask = gender == 'Male'
# Introduce bias: males slightly more likely to be positive class
bias_indices = male_mask & (np.random.random(2000) < 0.12)
y[bias_indices] = 1

# Split and train
X_train, X_test, y_train, y_test, gender_train, gender_test = train_test_split(
    X, y, gender, test_size=0.3, random_state=42
)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

print(f"Model trained on {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

### 2.1 Statistical Parity (Demographic Parity)

**Definition**: P(ŷ=1 | A=Male) should equal P(ŷ=1 | A=Female)

**When to use**: When you want equal representation in outcomes (e.g., marketing, university admissions)

**Limitation**: Ignores whether predictions are correct

In [ ]:
# Calculate statistical parity manually
male_positive_rate = y_pred[gender_test == 'Male'].mean()
female_positive_rate = y_pred[gender_test == 'Female'].mean()
stat_parity_diff = male_positive_rate - female_positive_rate

print("Statistical Parity Analysis:")
print(f"Male positive prediction rate: {male_positive_rate:.3f}")
print(f"Female positive prediction rate: {female_positive_rate:.3f}")
print(f"Difference: {stat_parity_diff:.3f}")
print(f"\nFair if difference < 0.05: {abs(stat_parity_diff) < 0.05}")

### 2.2 Disparate Impact Ratio (80% Rule)

**Definition**: P(ŷ=1 | Unprivileged) / P(ŷ=1 | Privileged) should be ≥ 0.80

**When to use**: Legal compliance (EEOC hiring guidelines), lending decisions

**Advantage**: Easy to interpret, legally established threshold

In [ ]:
# Calculate disparate impact
disparate_impact = female_positive_rate / male_positive_rate

print("Disparate Impact Analysis (80% Rule):")
print(f"Ratio: {disparate_impact:.3f}")
print(f"Passes 80% rule: {disparate_impact >= 0.80}")
print("\nInterpretation:")
if disparate_impact >= 0.80:
    print("✅ Model meets legal fairness standard")
else:
    print("⚠️ Model may face legal challenges")
    print(f"   Female rate needs to be at least {male_positive_rate * 0.80:.3f}")

### 2.3 Equal Opportunity

**Definition**: TPR should be equal across groups (among qualified individuals)

**When to use**: When false negatives are costly (loan approvals, medical diagnosis)

**Focus**: Qualified individuals should have equal chance of positive prediction

In [ ]:
# Calculate TPR (True Positive Rate) for each group
def calculate_tpr(y_true, y_pred, mask):
    y_true_masked = y_true[mask]
    y_pred_masked = y_pred[mask]
    
    # TPR = TP / (TP + FN) = TP / all actual positives
    positives = y_true_masked == 1
    if positives.sum() == 0:
        return 0.0
    true_positives = (y_pred_masked[positives] == 1).sum()
    return true_positives / positives.sum()

male_tpr = calculate_tpr(y_test, y_pred, gender_test == 'Male')
female_tpr = calculate_tpr(y_test, y_pred, gender_test == 'Female')
eq_opp_diff = abs(male_tpr - female_tpr)

print("Equal Opportunity Analysis:")
print(f"Male TPR (among qualified males): {male_tpr:.3f}")
print(f"Female TPR (among qualified females): {female_tpr:.3f}")
print(f"Difference: {eq_opp_diff:.3f}")
print(f"\nFair if difference < 0.05: {eq_opp_diff < 0.05}")
print("\nThis means qualified individuals of both genders")
print("have similar chances of being approved.")

### 2.4 Equalized Odds

**Definition**: Both TPR and FPR should be equal across groups

**When to use**: When both false positives AND false negatives matter (fraud detection, criminal justice)

**Most stringent**: Requires fairness for both qualified and unqualified individuals

In [ ]:
# Calculate FPR (False Positive Rate) for each group
def calculate_fpr(y_true, y_pred, mask):
    y_true_masked = y_true[mask]
    y_pred_masked = y_pred[mask]
    
    # FPR = FP / (FP + TN) = FP / all actual negatives
    negatives = y_true_masked == 0
    if negatives.sum() == 0:
        return 0.0
    false_positives = (y_pred_masked[negatives] == 1).sum()
    return false_positives / negatives.sum()

male_fpr = calculate_fpr(y_test, y_pred, gender_test == 'Male')
female_fpr = calculate_fpr(y_test, y_pred, gender_test == 'Female')
fpr_diff = abs(male_fpr - female_fpr)

# Equalized odds is max of TPR and FPR differences
eq_odds = max(eq_opp_diff, fpr_diff)

print("Equalized Odds Analysis:")
print(f"\nTPR difference: {eq_opp_diff:.3f}")
print(f"FPR difference: {fpr_diff:.3f}")
print(f"Equalized Odds (max): {eq_odds:.3f}")
print(f"\nFair if < 0.05: {eq_odds < 0.05}")
print("\nThis is the most comprehensive fairness metric,")
print("ensuring fairness for both groups in all scenarios.")

## Part 3: Using JusticeAI for Comprehensive Analysis

In [ ]:
# Let JusticeAI calculate all metrics automatically
evaluator = FairnessEvaluator(fairness_threshold=0.05)

report = evaluator.evaluate(
    model=model,
    X=X_test,
    y_true=y_test,
    sensitive_attrs=gender_test
)

# Get complete summary
summary = report.get_summary()

print("Complete Fairness Analysis:")
print(f"Overall Score: {summary['overall_score']:.1f}/100")
print(f"Passes Fairness: {summary['passes_fairness']}")
print(f"Violations: {summary['n_violations']}")
print(f"\nKey Metrics:")
print(f"  Statistical Parity Diff: {summary['statistical_parity_diff']:.4f}")
print(f"  Disparate Impact: {summary['disparate_impact_ratio']:.4f}")

## Part 4: Trade-offs Between Metrics

Due to the **Fairness Impossibility Theorem**, you cannot satisfy all metrics simultaneously.

In [ ]:
# Compare two models with different fairness profiles
print("Comparing Models with Different Fairness Trade-offs:\n")

# Model 1: High accuracy, potentially less fair
model1 = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
model1.fit(X_train, y_train)

# Model 2: Simpler model, potentially more fair
model2 = LogisticRegression(random_state=42, max_iter=1000)
model2.fit(X_train, y_train)

# Evaluate both
report1 = evaluator.evaluate(model1, X_test, y_test, gender_test)
report2 = evaluator.evaluate(model2, X_test, y_test, gender_test)

# Compare
from sklearn.metrics import accuracy_score

acc1 = accuracy_score(y_test, model1.predict(X_test))
acc2 = accuracy_score(y_test, model2.predict(X_test))

print(f"Random Forest:")
print(f"  Accuracy: {acc1:.3f}")
print(f"  Fairness Score: {report1.get_overall_score():.1f}/100")
print(f"\nLogistic Regression:")
print(f"  Accuracy: {acc2:.3f}")
print(f"  Fairness Score: {report2.get_overall_score():.1f}/100")

print("\n💡 Often there's a trade-off between model complexity and fairness!")

## Part 5: Choosing the Right Metric

Decision guide for metric selection:

### Metric Selection Guide

| Scenario | Recommended Metric | Why |
|----------|-------------------|-----|
| **Hiring** | Disparate Impact | Legal requirement (80% rule) |
| **Loan Approval** | Equal Opportunity | Don't deny qualified applicants |
| **Medical Diagnosis** | Equalized Odds | Both false pos/neg are critical |
| **Marketing** | Statistical Parity | Equal exposure to opportunities |
| **Risk Scoring** | Calibration | Probabilities must be accurate |
| **Criminal Justice** | Equalized Odds | High stakes for both types of errors |
| **University Admissions** | Equal Opportunity | Focus on qualified candidates |

### Key Questions to Ask:

1. **What are the consequences of false positives vs false negatives?**
   - If FN worse → Equal Opportunity
   - If both matter → Equalized Odds
   - If neither worse → Statistical Parity

2. **Do you need legal compliance?**
   - Yes → Disparate Impact (80% rule)

3. **Are base rates different between groups?**
   - Yes → Equal Opportunity or Equalized Odds
   - No → Statistical Parity okay

4. **Do you use probability scores?**
   - Yes → Also check Calibration

## Summary

In this tutorial, you learned:

✅ **Pre-training metrics**: Identify data bias before modeling  
✅ **Statistical Parity**: Equal outcomes across groups  
✅ **Disparate Impact**: Legal fairness standard (80% rule)  
✅ **Equal Opportunity**: Fairness for qualified individuals  
✅ **Equalized Odds**: Comprehensive fairness metric  
✅ **Trade-offs**: Cannot satisfy all metrics simultaneously  
✅ **Metric selection**: Choose based on your use case  

## Key Takeaways

1. **No single metric is perfect** - Use multiple metrics
2. **Context matters** - Choose metrics based on use case
3. **Trade-offs exist** - Balance fairness with other goals
4. **Monitor continuously** - Fairness can drift over time

## Next Steps

- **Tutorial 03**: Customize reports and interpret visualizations
- Explore [API documentation](https://justiceai-validation.github.io/JusticeAI/) for advanced features